In [8]:
%load_ext autoreload
%autoreload 2
%cd ..
import numpy as np
import matplotlib.pyplot as plt
from src.ortools_solver import CVRP_solver
from src.utils import random_choose_candidate_2
from src.QAP_solver import calculate_D, calculate_S_E, calculate_D_prime,add_depot,calculate_T
from src.optimization_methods import row_wise_optimization, greedy_scheduling
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/yq-mew/yq/chem1906/Projects


In [9]:
np.random.seed(0)
num_candidates = 2000
a = random_choose_candidate_2(96,96,num_candidates)
a[a>0] = 1
jobs = np.argwhere(a)

In [ ]:
# CVRP distance matrix
D_S = calculate_D(a.shape[0])
D_D = calculate_D(a.shape[1])
S, E = calculate_S_E(a)   
# calculate distance matrix
D_prime = calculate_D_prime(D_S,D_D, S, E)
D_prime = add_depot(D_prime)
VRP_distance, VRP_recorder = CVRP_solver(D_prime.astype(np.int64), solving_time =20)
print("VRP distance: ", VRP_distance)

In [ ]:
# calculate the cost of the non-optimized sequence
tasks = np.array(range(jobs.shape[0]))
tasks = tasks+1
# if tasks.shape[0] %8 != 0, pad with -1
if tasks.shape[0] %8 != 0:
    tasks = np.pad(tasks, (0, 8-tasks.shape[0]%8), 'constant', constant_values=-1)
unoptimized_seuqnece = tasks.reshape(-1, 8)
t = calculate_T(unoptimized_seuqnece)
d = D_prime[1:, 1:]
non_optimized_distance = np.trace(np.dot(t.T, d))
# change non_optimized_distance to integer
non_optimized_distance = int(non_optimized_distance)
print(f'non_optimized_distance: {non_optimized_distance}')

non_optimized_distance: 3083


In [ ]:
# calculate the cost of the row-wise optimized sequence
index_matrix = np.zeros((96,96))
for j in range(jobs.shape[0]):
    index_matrix[jobs[j, 0], jobs[j, 1]] = j+1
row_wise_optimized_sequence = row_wise_optimization(index_matrix)
if row_wise_optimized_sequence.shape[0] %8 != 0:
    row_wise_optimized_sequence = np.pad(row_wise_optimized_sequence, (0, 8-row_wise_optimized_sequence.shape[0]%8), 'constant', constant_values=-1)
row_wise_optimized_sequence = row_wise_optimized_sequence.reshape(-1, 8)
t = calculate_T(row_wise_optimized_sequence)
row_wise_optimized_distance = np.trace(np.dot(t.T, d))
# change non_optimized_distance to integer
row_wise_optimized_distance = int(row_wise_optimized_distance)
print(f'row_wise_optimized_distance: {row_wise_optimized_distance}')

row_wise_optimized_distance: 1776
